In [62]:
import numpy as np
import torch

In [63]:
# Units in microns
wavelength = 0.6  # fluorescence wavelength/um
# objective immersion media
n_measure = 1.33
# background refractive index, PDMS
n_b = 1.33
mag = 20  # Q：magnification有什么物理意义？
# pixel_size = 6.5e-6 感觉有点太小了 Q：这个pixel_size的单位是什么？
pixel_size = 4
na = 1
dx = pixel_size / mag
dy = dx
dz = 20 * pixel_size/mag
shape = [200,200,25]
np_ctype = "complex64"

In [64]:
y = (np.arange(shape[0]) - shape[0] // 2) * (dy)
x = (np.arange(shape[1]) - shape[1] // 2) * (dx)
xx, yy = np.meshgrid(x, y)
xx = np.array(xx)
yy = np.array(yy)

# spatial frequency sampling
uy = (np.arange(shape[0]) - shape[0] // 2) * (1 / dy / shape[0])
# print(uy)
ux = (np.arange(shape[1]) - shape[1] // 2) * (1 / dy / shape[1])
uxx, uyy = np.meshgrid(ux, uy)
uxx = np.array(uxx)
uyy = np.array(uyy)

In [65]:
uxx

array([[-2.5  , -2.475, -2.45 , ...,  2.425,  2.45 ,  2.475],
       [-2.5  , -2.475, -2.45 , ...,  2.425,  2.45 ,  2.475],
       [-2.5  , -2.475, -2.45 , ...,  2.425,  2.45 ,  2.475],
       ...,
       [-2.5  , -2.475, -2.45 , ...,  2.425,  2.45 ,  2.475],
       [-2.5  , -2.475, -2.45 , ...,  2.425,  2.45 ,  2.475],
       [-2.5  , -2.475, -2.45 , ...,  2.425,  2.45 ,  2.475]])

In [69]:
fx_source = 0.01
fy_source = -0.0146
fz_source = 0
fx_source_on_grid = np.round(fx_source*shape[1]/dx)*dx/shape[1]
fy_source_on_grid = np.round(fy_source*shape[0]/dy)*dy/shape[0]
fx_source = np.array(fx_source)
fy_source = np.array(fy_source)
fz_source = n_b/wavelength
ux_shifted = np.fft.ifftshift(xx)
uy_shifted = np.fft.ifftshift(yy)
uxx_shifted = np.fft.ifftshift(uxx)
uyy_shifted = np.fft.ifftshift(uyy)

In [70]:
a = (((ux_shifted-fx_source*shape[1])*np.conj(ux_shifted-fx_source*shape[1]))+((uy_shifted-fy_source*shape[0])*np.conj(uy_shifted-fy_source*shape[0])))**0.5

In [71]:
np.where(a,0,1)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [72]:
np.sum(np.where(a,0,1))

0

In [73]:
a

array([[3.53926546, 3.43021865, 3.3296246 , ..., 3.9097826 , 3.77973544,
        3.65600875],
       [3.70599514, 3.60199944, 3.50633712, ..., 4.06132983, 3.93629267,
        3.81764325],
       [3.87587409, 3.77655928, 3.68543078, ..., 4.21691831, 4.09663276,
        3.98276286],
       ...,
       [3.06307036, 2.93639234, 2.81822639, ..., 3.48459467, 3.33802337,
        3.19724882],
       [3.21720375, 3.0968371 , 2.98502931, ..., 3.62082863, 3.48      ,
        3.34520552],
       [3.37615166, 3.26165602, 3.15569327, ..., 3.76276494, 3.6274509 ,
        3.49834246]])

In [55]:
b = (n_b/wavelength)**2 - uxx_shifted*np.conj(uxx_shifted) - uyy_shifted*np.conj(uyy_shifted)

In [59]:
b[0,89]

-0.03701388888888868

In [57]:
b.shape

(200, 200)

In [60]:
c = b
c[c<0] = 0

In [68]:
for i in range(200):
    for j in range(200):
        if a[i,j]<0:
            print(i)
            print(j)